## 라이브러리 불러오기

In [1]:
import os ; os.chdir("C:/Users/user/UNet-project")
import numpy as np
from netCDF4 import Dataset

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import seaborn as sns

from torchvision import transforms, datasets

import matplotlib.pyplot as plt

import pandas as pd

# In[]
ckpt_dir = './checkpoint'

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

## 네트워크 구축하기

### class 정의하기

- 신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화함.

- nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력데이터에 대한 연산들을 구현함.

In [2]:
class UNet(nn.Module): 
    def __init__(self):
        super(UNet, self).__init__()
        
        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]
            
            cbr = nn.Sequential(*layers)
            
            return cbr
        
        # Contracting path
        self.enc1_1 = CBR2d(in_channels=4, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)
        
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        
        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)
        
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)
        
        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)
        
        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)
        
        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)
        
        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=2, stride=2, padding=0, bias=True)
        
        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)
    
        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)
    
    # 각각의 레이어 연결하기    
    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        
        pool1 = self.pool1(enc1_2)
        
        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2= self.pool2(enc2_2)
        
        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3= self.pool3(enc3_2)
        
        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4= self.pool4(enc4_2)
        
        enc5_1 = self.enc5_1(pool4)
        
        dec5_1 = self.dec5_1(enc5_1)
        
        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1) # dim = [0:batch, 1:channel, 2:height, 3:width]
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)
        
        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1) # dim = [0:batch, 1:channel, 2:height, 3:width]
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)
        
        unpool2 = self.unpool2(dec3_1)
        
        cat2 = torch.cat((unpool2, enc2_2), dim=1) # dim = [0:batch, 1:channel, 2:height, 3:width]
        dec2_2 = self.dec2_2(cat2)
        
        dec2_1 = self.dec2_1(dec2_2)
        
        unpool1 = self.unpool1(dec2_1)
        
        cat1 = torch.cat((unpool1, enc1_2), dim=1) # dim = [0:batch, 1:channel, 2:height, 3:width]
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)
        
        x = self.fc(dec1_1)
        
        return x
    

## bootstrap error
## 학습데이터 20개

In [ ]:
lr = 1e-3
batch_size = 10
num_epoch = 500
bootstrap_n = 1

train_len = 20

r2_list = []
nme_list = []
for fold in range(bootstrap_n):
    print(fold,'반복중')
    test_len = 119 - train_len

    all_list = list(range(70))  # 70개 중에서 부트스트랩하는 것으로
    train_list = list(np.random.choice(all_list,train_len))

    test_list = list(range(70,119)) #테스트셋은 뒤 49개로 고정
    print(train_list)
    # In[] SMOKE-CMAQ
    # CMAQ 연평균 PM농도
    pm2_5 = np.array([])

    for i in range(1,120):
        a = 'ACONC.' + str(i)
        nc = Dataset(a, 'r')
        pm2_5 = np.append(pm2_5, np.array([[[nc.variables['PM2_5']]]]))

    pm2_5 = np.reshape(pm2_5, [119,82,67])
    pm2_5 = pm2_5[:,8:-10,2:-1]
    pm2_5 = np.reshape(pm2_5, [119,64,64]) # convolution 연산을 위해

    pm = pm2_5


    y1 = np.array([])
    y2 = np.array([])
    y3 = np.array([])
    y4 = np.array([])
    y5 = np.array([])
    y6 = np.array([])

    for i in range(1,120):
        a = 'EMIS_AVG.' + str(i)
        nc = Dataset(a, 'r')
        y1 = np.append(y1, np.array([[[nc.variables['SO2']]]]))
        y2 = np.append(y2, np.array([[[nc.variables['PM2_5']]]]))
        y3 = np.append(y3, np.array([[[nc.variables['NOx']]]]))
        y4 = np.append(y4, np.array([[[nc.variables['VOCs']]]]))
        y5 = np.append(y5, np.array([[[nc.variables['NH3']]]]))
        y6 = np.append(y6, np.array([[[nc.variables['CO']]]]))

    y1 = np.reshape(y1, [119,82,67])
    y1 = y1[:,8:-10,2:-1]
    y1 = np.reshape(y1, [119,64,64])
    y1_max = np.max(y1)
    y1_min = np.min(y1)

    y2 = np.reshape(y2, [119,82,67])
    y2 = y2[:,8:-10,2:-1]
    y2 = np.reshape(y2, [119,64,64])
    y2_max = np.max(y2)
    y2_min = np.min(y2)

    y3 = np.reshape(y3, [119,82,67])
    y3 = y3[:,8:-10,2:-1]
    y3 = np.reshape(y3, [119,64,64])
    y3_max = np.max(y3)
    y3_min = np.min(y3)

    y4 = np.reshape(y4, [119,82,67])
    y4 = y4[:,8:-10,2:-1]
    y4 = np.reshape(y4, [119,64,64])
    y4_max = np.max(y4)
    y4_min = np.min(y4)

    y5 = np.reshape(y5, [119,82,67])
    y5 = y5[:,8:-10,2:-1]
    y5 = np.reshape(y5, [119,64,64])
    y5_max = np.max(y5)
    y5_min = np.min(y5)

    y6 = np.reshape(y6, [119,82,67])
    y6 = y6[:,8:-10,2:-1]
    y6 = np.reshape(y6, [119,64,64])


    def pm_data():
        a = np.concatenate((y3,y1,y4,y5), axis=0) # PM2.5, SO2, NH3
        a = np.reshape(a,[4,119,64,64]) # data generation for CNN ??
        a = np.transpose(a, (1,2,3,0)) # CNN 학습을 위한 순서변경
        print(np.shape(a))

        y = pm

        y = np.reshape(y, [119,64,64,1]) # CNN output data
        x_train = a[train_list]
        y_train = y[train_list]
        x_test = a[test_list]
        pm2_5 = y[test_list]
        return x_train, y_train, x_test, pm2_5, a, y


    pm_data1 = pm_data()
    x_train, y_train, x_test, pm2_5, a, y = pm_data1[0], pm_data1[1], pm_data1[2], pm_data1[3], pm_data1[4], pm_data1[5]#, pm_data1[6], pm_data1[7] 

    # In[]

    class TensorData(torch.utils.data.Dataset):
        def __init__(self, x_data, y_data):
            self.x_data = torch.FloatTensor(x_data)
            self.y_data = torch.FloatTensor(y_data)
            self.x_data = self.x_data.permute(0,3,1,2) # 이미지 개수, 채널 수, 너비, 높이
            self.y_data = self.y_data.permute(0,3,1,2)
            self.len = self.y_data.shape[0]

        def __getitem__(self, index):
            return self.x_data[index], self.y_data[index]

        def __len__(self):
            return self.len

    # 인스턴스(데이터) 생성
    train_data = TensorData(x_train, y_train)

    # 만들어진 데이터를 배치형태로 만들어주기위해 DataLoader에다가 넣어줌
    train_loader = DataLoader(train_data, batch_size = batch_size, shuffle=True)

    # In[] 데이터의 사이즈와 실제 데이터 확인해보기
    train_data[0][0].size()


    print(x_train.shape, torch.FloatTensor(x_train).permute(0,3,1,2).shape)
    # In[] train_loader안의 실제값 한개 뽑아서 확인
    dataiter = iter(train_loader)
    images, labels = dataiter.next()

    print(images.shape, labels.shape)


    # In[] 네트워크 생성하기
    net = UNet().to(device)

    # 손실함수 정의하기
    fn_loss = nn.MSELoss().to(device)

    # Optimizer 설정하기
    optim = torch.optim.Adam(net.parameters(), lr=lr)

    # 그밖의 부수적인 variables 설정하기
    num_data_train = len(x_train)

    num_batch_train = np.ceil(num_data_train / batch_size)

    # In[] 네트워크 저장하기
    def save(ckpt_dir, net, optim, epoch):
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)

        torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
                   "./%s/model_epoch%d.pth" % (ckpt_dir, epoch))

    # In[] 네트워크 불러오기
    def load(ckpt_dir, net, optim):
        if not os.path.exists(ckpt_dir):
            epoch = 0
            return net, optim, epoch

        ckpt_lst = os.listdir(ckpt_dir)
        ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

        dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

        net.load_state_dict(dict_model['net'])
        optim.load_state_dict(dict_model['optim'])
        epoch = int(ckpt_lst[-1].split('epoch')[1].split('pth')[0])

        return net, optim, epoch

    # In[] 네트워크 학습시키기

    st_epoch = 0
    #net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

    import time
    start = time.time()

    for epoch in range(st_epoch + 1, num_epoch + 1):
        net.train()
        loss_arr = []

        for batch, data in enumerate(train_loader, 1):
            # forward pass
            label = data[1].to(device)
            input = data[0].to(device)

            output = net(input)

            # backward pass
            optim.zero_grad()

            loss = fn_loss(output, label)
            loss.backward()

            optim.step()

            # 손실함수 계산
            loss_arr += [loss.item()]

            print('TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f' %
                  (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr)))

        if epoch % 50 == 0:
            save(ckpt_dir = ckpt_dir, net=net, optim=optim, epoch=epoch)


    print("time :", time.time() - start)

    # In[] r squared
    
    pre = net(torch.FloatTensor(x_test).permute(0,3,1,2).to(device))
    pre = np.reshape(pre.cpu().detach().numpy(),(len(x_test),64,64))

    def r_square(y_true, y_pred):
        SS_res = np.sum(np.square(y_true - y_pred))
        SS_tot = np.sum(np.square(y_true - np.mean(y_pred)))
        return (1-SS_res / SS_tot)
    # In[]
    def NME(y_true, y_pred):
        return np.sum( abs( (y_pred - y_true) / y_true ) ) / len(y_pred)


    # In[] r sq
    r_sq = []

    for i in range(len(pre)):
        r_sq.append(r_square(pre[i][::-1].reshape(64*64), pm2_5[i][::-1].reshape(64*64)))

    print(np.mean(r_sq),np.std(r_sq),np.min(r_sq))
    r2_list.append([np.mean(r_sq),np.std(r_sq),np.min(r_sq)])

    # In[]
    result = []

    for i in range(len(pre)):
        result.append(NME(pre[i][::-1].reshape(64*64), pm2_5[i][::-1].reshape(64*64)))
    
    nme_list.append(result)
#     # In[]
#     plt.figure(figsize=(12,6), dpi = 300)
#     plt.plot(np.arange(1,len(x_test),1), np.array(result) * 100)
#     plt.xlim([0,len(x_test)])
#     plt.ylim([0,15])
#     plt.plot([0,len(x_test)],[1,1])
#     plt.xlabel('Case number') ; plt.ylabel('Normalized Mean Error (NME %)')
#     # In[]
#     np.max(result)
#     np.mean(result)
#     np.std(result)

0 반복중
[41, 29, 9, 5, 5, 19, 3, 13, 51, 37, 58, 34, 61, 19, 25, 47, 34, 50, 66, 63]
(119, 64, 64, 4)
(20, 64, 64, 4) torch.Size([20, 4, 64, 64])
torch.Size([10, 4, 64, 64]) torch.Size([10, 1, 64, 64])
TRAIN: EPOCH 0001 / 0500 | BATCH 0001 / 0002 | LOSS 185.0751
TRAIN: EPOCH 0001 / 0500 | BATCH 0002 / 0002 | LOSS 183.1183
TRAIN: EPOCH 0002 / 0500 | BATCH 0001 / 0002 | LOSS 175.5044
TRAIN: EPOCH 0002 / 0500 | BATCH 0002 / 0002 | LOSS 175.2816
TRAIN: EPOCH 0003 / 0500 | BATCH 0001 / 0002 | LOSS 171.0160
TRAIN: EPOCH 0003 / 0500 | BATCH 0002 / 0002 | LOSS 169.3673
TRAIN: EPOCH 0004 / 0500 | BATCH 0001 / 0002 | LOSS 165.7203
TRAIN: EPOCH 0004 / 0500 | BATCH 0002 / 0002 | LOSS 164.1951
TRAIN: EPOCH 0005 / 0500 | BATCH 0001 / 0002 | LOSS 160.2699
TRAIN: EPOCH 0005 / 0500 | BATCH 0002 / 0002 | LOSS 159.9019
TRAIN: EPOCH 0006 / 0500 | BATCH 0001 / 0002 | LOSS 157.4452
TRAIN: EPOCH 0006 / 0500 | BATCH 0002 / 0002 | LOSS 156.5586
TRAIN: EPOCH 0007 / 0500 | BATCH 0001 / 0002 | LOSS 155.4151
TRAIN: 

TRAIN: EPOCH 0068 / 0500 | BATCH 0001 / 0002 | LOSS 68.2411
TRAIN: EPOCH 0068 / 0500 | BATCH 0002 / 0002 | LOSS 68.4919
TRAIN: EPOCH 0069 / 0500 | BATCH 0001 / 0002 | LOSS 67.5413
TRAIN: EPOCH 0069 / 0500 | BATCH 0002 / 0002 | LOSS 67.3022
TRAIN: EPOCH 0070 / 0500 | BATCH 0001 / 0002 | LOSS 66.0424
TRAIN: EPOCH 0070 / 0500 | BATCH 0002 / 0002 | LOSS 66.1133
TRAIN: EPOCH 0071 / 0500 | BATCH 0001 / 0002 | LOSS 65.0235
TRAIN: EPOCH 0071 / 0500 | BATCH 0002 / 0002 | LOSS 64.9397
TRAIN: EPOCH 0072 / 0500 | BATCH 0001 / 0002 | LOSS 64.3722
TRAIN: EPOCH 0072 / 0500 | BATCH 0002 / 0002 | LOSS 63.7983
TRAIN: EPOCH 0073 / 0500 | BATCH 0001 / 0002 | LOSS 62.3447
TRAIN: EPOCH 0073 / 0500 | BATCH 0002 / 0002 | LOSS 62.6675
TRAIN: EPOCH 0074 / 0500 | BATCH 0001 / 0002 | LOSS 61.3922
TRAIN: EPOCH 0074 / 0500 | BATCH 0002 / 0002 | LOSS 61.6035
TRAIN: EPOCH 0075 / 0500 | BATCH 0001 / 0002 | LOSS 61.0870
TRAIN: EPOCH 0075 / 0500 | BATCH 0002 / 0002 | LOSS 60.4585
TRAIN: EPOCH 0076 / 0500 | BATCH 0001 / 

TRAIN: EPOCH 0138 / 0500 | BATCH 0001 / 0002 | LOSS 13.1605
TRAIN: EPOCH 0138 / 0500 | BATCH 0002 / 0002 | LOSS 13.1200
TRAIN: EPOCH 0139 / 0500 | BATCH 0001 / 0002 | LOSS 12.6513
TRAIN: EPOCH 0139 / 0500 | BATCH 0002 / 0002 | LOSS 12.7093
TRAIN: EPOCH 0140 / 0500 | BATCH 0001 / 0002 | LOSS 12.5281
TRAIN: EPOCH 0140 / 0500 | BATCH 0002 / 0002 | LOSS 12.2616
TRAIN: EPOCH 0141 / 0500 | BATCH 0001 / 0002 | LOSS 12.1437
TRAIN: EPOCH 0141 / 0500 | BATCH 0002 / 0002 | LOSS 11.9065
TRAIN: EPOCH 0142 / 0500 | BATCH 0001 / 0002 | LOSS 11.2227
TRAIN: EPOCH 0142 / 0500 | BATCH 0002 / 0002 | LOSS 11.5030
TRAIN: EPOCH 0143 / 0500 | BATCH 0001 / 0002 | LOSS 11.2770
TRAIN: EPOCH 0143 / 0500 | BATCH 0002 / 0002 | LOSS 11.1185
TRAIN: EPOCH 0144 / 0500 | BATCH 0001 / 0002 | LOSS 10.9824
TRAIN: EPOCH 0144 / 0500 | BATCH 0002 / 0002 | LOSS 10.7537
TRAIN: EPOCH 0145 / 0500 | BATCH 0001 / 0002 | LOSS 10.4104
TRAIN: EPOCH 0145 / 0500 | BATCH 0002 / 0002 | LOSS 10.4021
TRAIN: EPOCH 0146 / 0500 | BATCH 0001 / 

TRAIN: EPOCH 0208 / 0500 | BATCH 0002 / 0002 | LOSS 0.9745
TRAIN: EPOCH 0209 / 0500 | BATCH 0001 / 0002 | LOSS 0.9444
TRAIN: EPOCH 0209 / 0500 | BATCH 0002 / 0002 | LOSS 0.9378
TRAIN: EPOCH 0210 / 0500 | BATCH 0001 / 0002 | LOSS 0.8732
TRAIN: EPOCH 0210 / 0500 | BATCH 0002 / 0002 | LOSS 0.8941
TRAIN: EPOCH 0211 / 0500 | BATCH 0001 / 0002 | LOSS 0.9180
TRAIN: EPOCH 0211 / 0500 | BATCH 0002 / 0002 | LOSS 0.8613
TRAIN: EPOCH 0212 / 0500 | BATCH 0001 / 0002 | LOSS 0.8288
TRAIN: EPOCH 0212 / 0500 | BATCH 0002 / 0002 | LOSS 0.8254
TRAIN: EPOCH 0213 / 0500 | BATCH 0001 / 0002 | LOSS 0.7697
TRAIN: EPOCH 0213 / 0500 | BATCH 0002 / 0002 | LOSS 0.7888
TRAIN: EPOCH 0214 / 0500 | BATCH 0001 / 0002 | LOSS 0.8058
TRAIN: EPOCH 0214 / 0500 | BATCH 0002 / 0002 | LOSS 0.7622
TRAIN: EPOCH 0215 / 0500 | BATCH 0001 / 0002 | LOSS 0.7125
TRAIN: EPOCH 0215 / 0500 | BATCH 0002 / 0002 | LOSS 0.7261
TRAIN: EPOCH 0216 / 0500 | BATCH 0001 / 0002 | LOSS 0.7565
TRAIN: EPOCH 0216 / 0500 | BATCH 0002 / 0002 | LOSS 0.69

TRAIN: EPOCH 0278 / 0500 | BATCH 0001 / 0002 | LOSS 0.0489
TRAIN: EPOCH 0278 / 0500 | BATCH 0002 / 0002 | LOSS 0.0499
TRAIN: EPOCH 0279 / 0500 | BATCH 0001 / 0002 | LOSS 0.0589
TRAIN: EPOCH 0279 / 0500 | BATCH 0002 / 0002 | LOSS 0.0484
TRAIN: EPOCH 0280 / 0500 | BATCH 0001 / 0002 | LOSS 0.0634
TRAIN: EPOCH 0280 / 0500 | BATCH 0002 / 0002 | LOSS 0.0537
TRAIN: EPOCH 0281 / 0500 | BATCH 0001 / 0002 | LOSS 0.0439
TRAIN: EPOCH 0281 / 0500 | BATCH 0002 / 0002 | LOSS 0.0446
TRAIN: EPOCH 0282 / 0500 | BATCH 0001 / 0002 | LOSS 0.0524
TRAIN: EPOCH 0282 / 0500 | BATCH 0002 / 0002 | LOSS 0.0428
TRAIN: EPOCH 0283 / 0500 | BATCH 0001 / 0002 | LOSS 0.0504
TRAIN: EPOCH 0283 / 0500 | BATCH 0002 / 0002 | LOSS 0.0399
TRAIN: EPOCH 0284 / 0500 | BATCH 0001 / 0002 | LOSS 0.0271
TRAIN: EPOCH 0284 / 0500 | BATCH 0002 / 0002 | LOSS 0.0534
TRAIN: EPOCH 0285 / 0500 | BATCH 0001 / 0002 | LOSS 0.0366
TRAIN: EPOCH 0285 / 0500 | BATCH 0002 / 0002 | LOSS 0.0393
TRAIN: EPOCH 0286 / 0500 | BATCH 0001 / 0002 | LOSS 0.05

...

TRAIN: EPOCH 0363 / 0500 | BATCH 0001 / 0002 | LOSS 0.0133
TRAIN: EPOCH 0363 / 0500 | BATCH 0002 / 0002 | LOSS 0.0151
TRAIN: EPOCH 0364 / 0500 | BATCH 0001 / 0002 | LOSS 0.0103
TRAIN: EPOCH 0364 / 0500 | BATCH 0002 / 0002 | LOSS 0.0138
TRAIN: EPOCH 0365 / 0500 | BATCH 0001 / 0002 | LOSS 0.0153
TRAIN: EPOCH 0365 / 0500 | BATCH 0002 / 0002 | LOSS 0.0159
TRAIN: EPOCH 0366 / 0500 | BATCH 0001 / 0002 | LOSS 0.0093
TRAIN: EPOCH 0366 / 0500 | BATCH 0002 / 0002 | LOSS 0.0113
TRAIN: EPOCH 0367 / 0500 | BATCH 0001 / 0002 | LOSS 0.0090
TRAIN: EPOCH 0367 / 0500 | BATCH 0002 / 0002 | LOSS 0.0118
TRAIN: EPOCH 0368 / 0500 | BATCH 0001 / 0002 | LOSS 0.0112
TRAIN: EPOCH 0368 / 0500 | BATCH 0002 / 0002 | LOSS 0.0138
TRAIN: EPOCH 0369 / 0500 | BATCH 0001 / 0002 | LOSS 0.0104
TRAIN: EPOCH 0369 / 0500 | BATCH 0002 / 0002 | LOSS 0.0097
TRAIN: EPOCH 0370 / 0500 | BATCH 0001 / 0002 | LOSS 0.0220
TRAIN: EPOCH 0370 / 0500 | BATCH 0002 / 0002 | LOSS 0.0249
TRAIN: EPOCH 0371 / 0500 | BATCH 0001 / 0002 | LOSS 0.01

TRAIN: EPOCH 0434 / 0500 | BATCH 0001 / 0002 | LOSS 0.0164
TRAIN: EPOCH 0434 / 0500 | BATCH 0002 / 0002 | LOSS 0.0115
TRAIN: EPOCH 0435 / 0500 | BATCH 0001 / 0002 | LOSS 0.0148
TRAIN: EPOCH 0435 / 0500 | BATCH 0002 / 0002 | LOSS 0.0115
TRAIN: EPOCH 0436 / 0500 | BATCH 0001 / 0002 | LOSS 0.0161
TRAIN: EPOCH 0436 / 0500 | BATCH 0002 / 0002 | LOSS 0.0159
TRAIN: EPOCH 0437 / 0500 | BATCH 0001 / 0002 | LOSS 0.0241
TRAIN: EPOCH 0437 / 0500 | BATCH 0002 / 0002 | LOSS 0.0249
TRAIN: EPOCH 0438 / 0500 | BATCH 0001 / 0002 | LOSS 0.0092
TRAIN: EPOCH 0438 / 0500 | BATCH 0002 / 0002 | LOSS 0.0132
TRAIN: EPOCH 0439 / 0500 | BATCH 0001 / 0002 | LOSS 0.0099
TRAIN: EPOCH 0439 / 0500 | BATCH 0002 / 0002 | LOSS 0.0127
TRAIN: EPOCH 0440 / 0500 | BATCH 0001 / 0002 | LOSS 0.0123
TRAIN: EPOCH 0440 / 0500 | BATCH 0002 / 0002 | LOSS 0.0122
TRAIN: EPOCH 0441 / 0500 | BATCH 0001 / 0002 | LOSS 0.0156
TRAIN: EPOCH 0441 / 0500 | BATCH 0002 / 0002 | LOSS 0.0229
TRAIN: EPOCH 0442 / 0500 | BATCH 0001 / 0002 | LOSS 0.01

TRAIN: EPOCH 0003 / 0500 | BATCH 0001 / 0002 | LOSS 158.2845
TRAIN: EPOCH 0003 / 0500 | BATCH 0002 / 0002 | LOSS 158.4530
TRAIN: EPOCH 0004 / 0500 | BATCH 0001 / 0002 | LOSS 156.4846
TRAIN: EPOCH 0004 / 0500 | BATCH 0002 / 0002 | LOSS 154.3462
TRAIN: EPOCH 0005 / 0500 | BATCH 0001 / 0002 | LOSS 151.0957
TRAIN: EPOCH 0005 / 0500 | BATCH 0002 / 0002 | LOSS 151.1522
TRAIN: EPOCH 0006 / 0500 | BATCH 0001 / 0002 | LOSS 148.1968
TRAIN: EPOCH 0006 / 0500 | BATCH 0002 / 0002 | LOSS 148.5027
TRAIN: EPOCH 0007 / 0500 | BATCH 0001 / 0002 | LOSS 145.2262
TRAIN: EPOCH 0007 / 0500 | BATCH 0002 / 0002 | LOSS 146.1249
TRAIN: EPOCH 0008 / 0500 | BATCH 0001 / 0002 | LOSS 146.5874
TRAIN: EPOCH 0008 / 0500 | BATCH 0002 / 0002 | LOSS 144.1172
TRAIN: EPOCH 0009 / 0500 | BATCH 0001 / 0002 | LOSS 143.7195
TRAIN: EPOCH 0009 / 0500 | BATCH 0002 / 0002 | LOSS 142.2781
TRAIN: EPOCH 0010 / 0500 | BATCH 0001 / 0002 | LOSS 141.7013
TRAIN: EPOCH 0010 / 0500 | BATCH 0002 / 0002 | LOSS 140.4794
TRAIN: EPOCH 0011 / 0500

TRAIN: EPOCH 0072 / 0500 | BATCH 0002 / 0002 | LOSS 49.0372
TRAIN: EPOCH 0073 / 0500 | BATCH 0001 / 0002 | LOSS 48.2587
TRAIN: EPOCH 0073 / 0500 | BATCH 0002 / 0002 | LOSS 47.8149
TRAIN: EPOCH 0074 / 0500 | BATCH 0001 / 0002 | LOSS 47.6966
TRAIN: EPOCH 0074 / 0500 | BATCH 0002 / 0002 | LOSS 46.6614
TRAIN: EPOCH 0075 / 0500 | BATCH 0001 / 0002 | LOSS 44.7545
TRAIN: EPOCH 0075 / 0500 | BATCH 0002 / 0002 | LOSS 45.4744
TRAIN: EPOCH 0076 / 0500 | BATCH 0001 / 0002 | LOSS 44.8089
TRAIN: EPOCH 0076 / 0500 | BATCH 0002 / 0002 | LOSS 44.3478
TRAIN: EPOCH 0077 / 0500 | BATCH 0001 / 0002 | LOSS 43.7713
TRAIN: EPOCH 0077 / 0500 | BATCH 0002 / 0002 | LOSS 43.1933
TRAIN: EPOCH 0078 / 0500 | BATCH 0001 / 0002 | LOSS 42.4519
TRAIN: EPOCH 0078 / 0500 | BATCH 0002 / 0002 | LOSS 42.0782
TRAIN: EPOCH 0079 / 0500 | BATCH 0001 / 0002 | LOSS 40.8760
TRAIN: EPOCH 0079 / 0500 | BATCH 0002 / 0002 | LOSS 40.9631
TRAIN: EPOCH 0080 / 0500 | BATCH 0001 / 0002 | LOSS 40.6743
TRAIN: EPOCH 0080 / 0500 | BATCH 0002 / 

TRAIN: EPOCH 0142 / 0500 | BATCH 0002 / 0002 | LOSS 3.7430
TRAIN: EPOCH 0143 / 0500 | BATCH 0001 / 0002 | LOSS 3.4308
TRAIN: EPOCH 0143 / 0500 | BATCH 0002 / 0002 | LOSS 3.5579
TRAIN: EPOCH 0144 / 0500 | BATCH 0001 / 0002 | LOSS 3.3869
TRAIN: EPOCH 0144 / 0500 | BATCH 0002 / 0002 | LOSS 3.3929
TRAIN: EPOCH 0145 / 0500 | BATCH 0001 / 0002 | LOSS 3.3513
TRAIN: EPOCH 0145 / 0500 | BATCH 0002 / 0002 | LOSS 3.2218
TRAIN: EPOCH 0146 / 0500 | BATCH 0001 / 0002 | LOSS 3.2342
TRAIN: EPOCH 0146 / 0500 | BATCH 0002 / 0002 | LOSS 3.0641
TRAIN: EPOCH 0147 / 0500 | BATCH 0001 / 0002 | LOSS 3.0071
TRAIN: EPOCH 0147 / 0500 | BATCH 0002 / 0002 | LOSS 2.9031
TRAIN: EPOCH 0148 / 0500 | BATCH 0001 / 0002 | LOSS 2.7577
TRAIN: EPOCH 0148 / 0500 | BATCH 0002 / 0002 | LOSS 2.7661
TRAIN: EPOCH 0149 / 0500 | BATCH 0001 / 0002 | LOSS 2.6488
TRAIN: EPOCH 0149 / 0500 | BATCH 0002 / 0002 | LOSS 2.6353
TRAIN: EPOCH 0150 / 0500 | BATCH 0001 / 0002 | LOSS 2.5290
TRAIN: EPOCH 0150 / 0500 | BATCH 0002 / 0002 | LOSS 2.49

In [ ]:
pre = net(torch.FloatTensor(x_test).permute(0,3,1,2).to(device))
pre = np.reshape(pre.cpu().detach().numpy(),(len(x_test),64,64))


In [ ]:
pm2_5.shape

In [ ]:
pre.reshape(-1)

In [ ]:
print(np.mean(r_sq),np.std(r_sq),np.min(r_sq))

In [ ]:
plt.plot(pm2_5[:,:,:,:].reshape(-1),pre[:,:,:].reshape(-1),'bo',markersize=1)
plt.plot([0,70],[0,70])
plt.savefig(f"plots/bootstrapresult2/bootstrap{bootstrap_n}_train{train_len}_epoch{num_epoch}_batchsize{batch_size}_1to1plot.png")
plt.close()

In [ ]:
result_df = pd.DataFrame(r2_list,columns = ['mean_r2', 'std_r2', 'min_r2'])
result_df.loc[:,'nme'] = [np.mean(np.array(nme_list[i]) * 100) for i in range(len(nme_list))]
result_df.to_csv(f"plots/bootstrapresult2/bootstrap{bootstrap_n}_train{train_len}_epoch{num_epoch}_batchsize{batch_size}_r2.csv")

plt.figure(figsize=(12,6), dpi = 300)

for i in range(len(nme_list)):
    print(i)
    plt.plot(np.arange(1,len(nme_list[i])+1,1), np.array(nme_list[i]) * 100)


plt.xlim([0,len(x_test)])
plt.ylim([0,15])



boot_mean = np.mean([np.mean(np.array(nme_list[i]) * 100) for i in range(len(nme_list))])
boot_std = np.std([np.mean(np.array(nme_list[i]) * 100) for i in range(len(nme_list))])
upper = boot_mean + (1.960 * boot_std / (bootstrap_n)**0.5)
lower = boot_mean - (1.960 * boot_std / (bootstrap_n)**0.5)

plt.plot([0,len(x_test)],[boot_mean,boot_mean], label = 'bootstrap_mean')

plt.fill_betweenx([lower,upper],[len(x_test),len(x_test)], alpha = 0.5, label='bootstraped_testerror_0.95')

plt.xlabel('Case number') ; plt.ylabel('Normalized Mean Error (NME %)')
plt.legend()

plt.savefig(f"plots/bootstrapresult2/bootstrap{bootstrap_n}_train{train_len}_epoch{num_epoch}_batchsize{batch_size}_nme.png")
plt.close()


plt.figure(figsize=(12,6), dpi = 300)
plt.hist([np.mean(np.array(nme_list[i]) * 100) for i in range(len(nme_list))])

plt.figure(figsize=(12,6), dpi = 300)
plt.hist([np.mean(np.array(nme_list[i]) * 100) for i in range(len(nme_list))])
plt.vlines(boot_mean, 0,2, color='gray', linestyle='solid', linewidth=3, label = 'boot_mean')
plt.vlines(upper, 0,2, color='gray', linestyle='solid', linewidth=3, label = 'boot_testerror_0.975upper')
plt.vlines(lower, 0,2, color='gray', linestyle='solid', linewidth=3, label = 'boot_testerror_0.125lower')
plt.legend()

plt.savefig(f"plots/bootstrapresult2/bootstrap{bootstrap_n}_train{train_len}_epoch{num_epoch}_batchsize{batch_size}_boots_hist.png")
plt.close()